Este trecho do código é responsável por toda a operação diária do robô de negociação. Ele inicia a conexão com a plataforma, carrega os ativos que serão monitorados e entra em um ciclo contínuo até o horário limite definido. Durante esse período, o sistema acompanha o mercado em tempo real, posiciona ordens no book, ajusta preços quando necessário e executa as operações conforme os critérios estabelecidos.

Quando ainda não há trade aberto, o robô monta automaticamente o book inicial, distribuindo ordens de compra e venda para buscar a melhor entrada. Depois que uma operação é acionada, o foco passa a ser gestão de risco: o código monitora o movimento dos preços, reduz exposição quando o mercado perde liquidez, ativa stop gain em momentos favoráveis e encerra posições rapidamente caso haja risco de perda acima do esperado.

Em termos simples, este bloco automatiza todo o processo de tomada de decisão intraday: entrar na operação, monitorar condições, ajustar ordens e proteger o capital. Ele garante disciplina, rapidez de execução e redução de erros humanos — exatamente o que se espera de um sistema profissional de trading.

In [ ]:
# BIBLIOTECAS
import time
import pytz
import pandas as pd
import yfinance as yf
import MetaTrader5 as mt5
from datetime import datetime
from datetime import datetime, timedelta, timezone

In [1]:
expiration_time = datetime.now() + timedelta(minutes=30)
expiration_timestamp = int(expiration_time.timestamp())

# DEFINIÇÃO DE PARAMETROS DOS ATIVOS
def ativos_parametros(symbol1, symbol2):
    # PREÇOS
    lot1 = 100
    symbol1_ask = mt5.symbol_info_tick(symbol1).ask
    symbol1_bid = mt5.symbol_info_tick(symbol1).bid
    symbol1_last = mt5.symbol_info_tick(symbol1).last
    return symbol1_ask, symbol1_bid, symbol1_last, lot1

# VERIFICAR DISPONIBILIDADE DOS ATIVOS
def verificar_disponibilidade_ativos():
    symbol1 = 'PETRK332'
    symbol2 = 'PETRK370'

    while True:
        symbol1_info = mt5.symbol_info(symbol1)
        symbol2_info = mt5.symbol_info(symbol2)
        
        if symbol1_info is None:
            print(symbol1, 'Ativo 1 não encontrado')
        elif not symbol1_info.visible:
            print(symbol1, 'Ativo 1 não visível')
            if not mt5.symbol_select(symbol1, True):
                print(f'Symbol_select({symbol1}) falhou, tentando novamente...')
                time.sleep(2)  # Espera um pouco antes de tentar novamente
                continue  # Continua o loop para rechecagem

        if symbol2_info is None:
            print(symbol2, 'Ativo 2 não encontrado')
        elif not symbol2_info.visible:
            print(symbol2, 'Ativo 2 não visível')
            if not mt5.symbol_select(symbol2, True):
                print(f'Symbol_select({symbol2}) falhou, tentando novamente...')
                time.sleep(2)  # Espera um pouco antes de tentar novamente
                continue  # Continua o loop para rechecagem

        # Se ambos os ativos estão visíveis, sai do loop
        if symbol1_info and symbol1_info.visible and symbol2_info and symbol2_info.visible:
            print('Ambos os ativos estão visíveis')
            return symbol1, symbol2
            
    return symbol1, symbol2

# COLETAR TRADES ABERTOS
def get_trades():
    trades = mt5.positions_get(symbol=symbol1)
    if trades is None:
        print("Nenhum trade: {}".format(mt5.last_error()))
        return None
    trade_data = []
    for trade in trades:
        trade_dict = {
            'ticket': trade.ticket,'time': trade.time,'type': trade.type,'volume': trade.volume,
            'price_open': trade.price_open,'sl': trade.sl,'tp': trade.tp,'price_current': trade.price_current,
            'swap': trade.swap,'profit': trade.profit,'symbol': trade.symbol,'comment': trade.comment,'external_id': trade.external_id}
        trade_data.append(trade_dict)
    trades = pd.DataFrame(trade_data)
    return trades

# COLETAR ORDENS AGENDADAS 
def get_agendadas():
    agendadas = mt5.orders_get(symbol=symbol1)
    agendadas_data = []
    for agendada in agendadas:
        agendada_dict = {
            'ticket': agendada.ticket,'time_setup': agendada.time_setup,'time_done': agendada.time_done,
            'type': agendada.type,'volume_initial': agendada.volume_initial,'volume_current': agendada.volume_current,
            'price_open': agendada.price_open,'price_current': agendada.price_current,'symbol': agendada.symbol,
            'comment': agendada.comment,'external_id': agendada.external_id}
        agendadas_data.append(agendada_dict)
        agendadas = pd.DataFrame(agendadas_data)
        is_symbol1_present = agendadas['symbol'] == symbol1
        agendadas = agendadas[is_symbol1_present]
    return agendadas
    
# CANCELAR STOP GAIN TRADES ABERTOS
def cancel_stop_gain_trading(stopgain_opendtrade):
    position_id = stopgain_opendtrade.order
    close_zero_order={
        "order": position_id,
        "action": mt5.TRADE_ACTION_REMOVE}
    mt5.order_send(close_zero_order)
    print('Stop gain de trade aberto cancelado')
    global stop_gain_trading
    if stop_gain_trading is True:
        stop_gain_trading = False

# CANCELAR STOP GAIN ORDENS AGENDADAS
def cancel_stop_gain_pending(request_stop_gain):
    position_id = request_stop_gain.order
    close_zero_order={
        "order": position_id,
        "action": mt5.TRADE_ACTION_REMOVE,}
    mt5.order_send(close_zero_order)
    print('Stop gain de ordem pendente cancelado')
    global stop_gain_pending
    if stop_gain_pending is True:
        stop_gain_pending = False

# ZERAGEM COMPRA
def buy_stop_market():
    trades = get_trades()
    if len(trades) > 0:
        point = mt5.symbol_info(symbol1).point
        price = mt5.symbol_info_tick(symbol1).last
        deviation = 20
        close_stop_order = {
            "action": mt5.TRADE_ACTION_DEAL,"symbol": symbol1,"volume": float(lot1),"type": mt5.ORDER_TYPE_SELL,
            "price": price,"deviation": deviation,"magic": 234000,"comment": "Zeragem mercado","type_time": mt5.ORDER_TIME_GTC,
            "expiration": expiration_time,"type_filling": mt5.ORDER_FILLING_RETURN,}
        result = mt5.order_send(close_stop_order)
        print('A posição comprada foi zerada')
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("2. order_send failed, retcode={}".format(result.retcode))
            result_dict=result._asdict()
            for field in result_dict.keys():
                print("   {}={}".format(field,result_dict[field]))
                if field=="request":
                    traderequest_dict=result_dict[field]._asdict()
                    for tradereq_filed in traderequest_dict:
                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
            print("shutdown() and quit")
            mt5.shutdown()
            quit()
            
# ZERAGEM VENDA
def sell_stop_market():
    trades = get_trades()
    if len(trades) > 0:
        point = mt5.symbol_info(symbol1).point
        price = mt5.symbol_info_tick(symbol1).last
        deviation = 20
        close_stop_order = {
            "action": mt5.TRADE_ACTION_DEAL,"symbol": symbol1,"volume": float(lot1),"type": mt5.ORDER_TYPE_BUY,
            "price": price,"deviation": deviation,"magic": 234000,"comment": "Zeragem mercado","type_time": mt5.ORDER_TIME_GTC,
            "expiration": expiration_time,"type_filling": mt5.ORDER_FILLING_RETURN,}
        result = mt5.order_send(close_stop_order)
        print('A posição vendida foi zerada')
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("2. order_send failed, retcode={}".format(result.retcode))
            result_dict=result._asdict()
            for field in result_dict.keys():
                print("   {}={}".format(field,result_dict[field]))
                if field=="request":
                    traderequest_dict=result_dict[field]._asdict()
                    for tradereq_filed in traderequest_dict:
                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
            print("shutdown() and quit")
            mt5.shutdown()
            quit()

# PEGAR BOOK DE OFERTAS
def spread_book():
    if mt5.market_book_add(symbol1):
        items = mt5.market_book_get(symbol1)
        databook = pd.DataFrame(items)
    else:
        print("mt5.market_book_add failed, error code =",mt5.last_error())
    spread = databook.iloc[4:6]
    return spread

# PEGAR BOOK DE OFERTAS COMPLETO
def spread_book_2():
    if mt5.market_book_add(symbol1):
        items = mt5.market_book_get(symbol1)
        databook = pd.DataFrame(items)
    else:
        print("mt5.market_book_add failed, error code =",mt5.last_error())
    spread_2  = databook
    return spread_2

# VERIFICAÇÃO DE HORÁRIO
def is_time_exceeded():
    import datetime
    current_time = datetime.datetime.now().time()
    target_time = datetime.time(16, 40)
    if current_time >= target_time:
        return True
    else:
        return False

# ORDENAR LISTA DE PREÇOS
def ordenar(prices_list, symbol1_last):
    return sorted(prices_list, key=lambda x: abs(x - symbol1_last))
    
# ADICIONAR ORDENS NAS LACUNAS DO BOOK
def ordem_lacuna(lista_ordens):                                 
    ''' Verificar se book > 100k para adicionar ordem na lacuna '''
    agendadas = get_agendadas()
    spread = spread_book_2()
    prices_list = spread.iloc[:, 1].tolist()
    symbol1_bid = mt5.symbol_info_tick(symbol1).bid
    symbol1_ask = mt5.symbol_info_tick(symbol1).ask

    if symbol1_ask in prices_list:
        prices_list.remove(symbol1_ask)
    if symbol1_bid in prices_list:
        prices_list.remove(symbol1_bid)

    prices_list = [value for value in prices_list if value not in agendadas['price_open'].values]
    prices_list = ordenar(prices_list, symbol1_last)
    
    # atualizar lista de ordens abertas
    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
    lista_ordens = list(set(lista_ordens))
    # if i != 0.01 and i != symbol1_last and i != (symbol1_last + 0.01) and i != (symbol1_last - 0.01):
    for i in prices_list:
        try:
            if i > 0.01:
                index = spread.index[spread.iloc[:, 1] == i].tolist()  # Correção: Usar 'i' em vez de 'price'
                if index:  # Verifica se a lista 'index' não está vazia
                    value = spread.iloc[index[0], 2]  # Index[0] para pegar o primeiro índice, 2 para a terceira coluna
                    if value > 30000:
                        if i > symbol1_last:
                            trade = 'Venda'
                            print('Ordem de venda adicionada na lacuna do book...')
                            type_to_send = mt5.ORDER_TYPE_SELL_LIMIT
                        elif i < symbol1_last:
                            trade = 'Compra'
                            print('Ordem de compra adicionada na lacuna do book...')
                            type_to_send = mt5.ORDER_TYPE_BUY_LIMIT
                        price = round(i,2)
                        agend_order = {'action': mt5.TRADE_ACTION_PENDING,'symbol': symbol1,'volume': float(lot1),
                            'type': type_to_send,'price': price,'deviation': deviation,'magic': 234000,
                            'comment': trade,'type_time': mt5.ORDER_TIME_GTC,"expiration": expiration_time,'type_filling': mt5.ORDER_FILLING_RETURN,}
                        agend_order = mt5.order_send(agend_order)
                        if agend_order is None:
                            print("Erro ao enviar ordem: ", mt5.last_error())
                        else:
                            if agend_order.retcode != mt5.TRADE_RETCODE_DONE:
                                print("order_send falhou, retcode: {}".format(agend_order.retcode))
                                time.sleep(5)
                            elif agend_order.retcode == mt5.TRADE_RETCODE_DONE:
                                lista_ordens.append(agend_order)
                            
                                agendadas = get_agendadas()
                                lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                lista_ordens = list(set(lista_ordens))
                        #print(lista_ordens)
                else:
                    print("Nenhum índice encontrado para o preço:", i)

        except UnboundLocalError:
            pass

# ADICIONAR ORDEM STOP GAIN PARA POSIÇÃO COMPRADA
def add_stopgain_compra(price_stop_gain):
    deviation = 5
    request_stop_gain = {'action': mt5.TRADE_ACTION_PENDING,'symbol': symbol1,'volume': float(lot1),        
                'type': mt5.ORDER_TYPE_SELL_LIMIT,'price': price_stop_gain,'deviation': deviation,'magic': 234000,
                'comment': 'Stop Gain','type_time': mt5.ORDER_TIME_GTC,"expiration": expiration_time,
                'type_filling': mt5.ORDER_FILLING_RETURN,}
    request_stop_gain = mt5.order_send(request_stop_gain)
    if request_stop_gain.retcode != mt5.TRADE_RETCODE_DONE:
        print("2. order_send failed, retcode={}".format(request_stop_gain.retcode))
        result_dict=request_stop_gain._asdict()
        for field in result_dict.keys():
            print("   {}={}".format(field,result_dict[field]))
            if field=="request":
                traderequest_dict=result_dict[field]._asdict()
                for tradereq_filed in traderequest_dict:
                    print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
    print('Stop Gain de ordem pendente de compra adicionado')
    stop_gain_pending = True
    return request_stop_gain

# ADICIONAR ORDEM STOP GAIN PARA POSIÇÃO VENDIDA
def add_stopgain_venda(price_stop_gain):
    deviation = 5
    request_stop_gain = {'action': mt5.TRADE_ACTION_PENDING,'symbol': symbol1,'volume': float(lot1),
                'type': mt5.ORDER_TYPE_BUY_LIMIT,'price': price_stop_gain,'deviation': deviation,'magic': 234000,
                'comment': 'Stop Gain','type_time': mt5.ORDER_TIME_GTC,"expiration": expiration_time,
                'type_filling': mt5.ORDER_FILLING_RETURN,}
    request_stop_gain = mt5.order_send(request_stop_gain)
    if request_stop_gain.retcode != mt5.TRADE_RETCODE_DONE:
        print("2. order_send failed, retcode={}".format(request_stop_gain.retcode))
        result_dict=request_stop_gain._asdict()
        for field in result_dict.keys():
            print("   {}={}".format(field,result_dict[field]))
            if field=="request":
                traderequest_dict=result_dict[field]._asdict()
                for tradereq_filed in traderequest_dict:
                    print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))     
    print('Stop Gain de ordem pendente de venda adicionado')
    stop_gain_pending = True
    return request_stop_gain

# EXCLUIR ORDENS E TRADES EM ABERTO
def encerramento_pregao(lista_ordens,stopgain_opendtrade):
    global stop_gain_pending
    lista_to_end = []
    agendadas = get_agendadas()
    for ordem in lista_ordens:
        price = round(ordem.request.price,2)
        if price in agendadas['price_open'].values:
            lista_to_end.append(ordem)
    display(agendadas)
    display(lista_to_end)

    # CANCELAR AS AGENDADAS
    for ordem in lista_to_end:
        position_id = ordem.order
        position_to_cancel={"order": position_id,"action": mt5.TRADE_ACTION_REMOVE}
        mt5.order_send(position_to_cancel)

    # STOPAR TRADE ABERTO, CASO TENHA
    trades = get_trades()
    if not trades.empty and 'type' in trades.columns:
        type_trade = round(trades['type'][0],2)

        # VERIFICA SE É TRADE DE VENDA
        if type_trade == 1:
            symbol1_last = mt5.symbol_info_tick(symbol1).last
            spread = spread_book()
            trades = get_trades()
            if len(trades) > 0:
                stop = sell_stop_market()
                trades = get_trades()
                print('Stop - Fim do pregrão')
                if stop_gain_pending is True and len(trades) == 0:
                    cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                elif stop_gain_trading is True and len(trades) == 0:
                    cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)

        # VERIFICA SE É TRADE DE COMPRA
        elif type_trade == 0:
            symbol1_last = mt5.symbol_info_tick(symbol1).last
            spread = spread_book()
            stop = buy_stop_market()
            trades = get_trades()
            print('Stop - Fim do pregrão')
            if stop_gain_pending is True and len(trades) == 0:
                cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
            elif stop_gain_trading is True and len(trades) == 0:
                cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)

    # CANCELAR STOP GAIN DE ORDENS PENDENTES E TRADING
    if stop_gain_pending is True:
        cancel = cancel_stop_gain_pending(request_stop_gain)
        stop_gain_pending = False

    if stop_gain_trading is True:
        cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
        stop_gain_trading = False

    print('Horário limite para operações, ordens sendo canceladas e código encerrado...')

# VERIFICAÇÃO DE STOP GAIN PENDING 1   
def stop_gain_pending_verificacao(request_stop_gain):
    if stop_gain_pending is True: 
        cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)

# VERIFICAÇÃO DE STOP GAIN TRADING 1   
def stop_gain_trading_verificacao(stopgain_opendtrade):
    if stop_gain_pending is True: 
        cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
    elif stop_gain_trading is True and len(trades) == 0:
        cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)

# STOPAR CASO TRADE ABERTO SEJA STOP GAIN       
def stop_gain_trading_stop():
    try:
        trades = get_trades()
        if len(trades) > 0:
            fake_trade = any(trades['comment'].str.contains('Stop Gain', case=False))
            fake_trade_1 = any(trades['comment'].str.contains('Zeragem mercado', case=False))
            if fake_trade or fake_trade_1:
                type_trade = round(trades['type'][0],2)
                # VERIFICA SE É TRADE DE COMPRA
                if type_trade == 0:
                    point = mt5.symbol_info(symbol1).point
                    price = mt5.symbol_info_tick(symbol1).last
                    deviation = 20
                    close_stop_order = {
                        "action": mt5.TRADE_ACTION_DEAL,"symbol": symbol1,"volume": float(lot1),"type": mt5.ORDER_TYPE_SELL,
                        "price": price,"deviation": deviation,"magic": 234000,"comment": "Zeragem mercado","type_time": mt5.ORDER_TIME_GTC,
                        "expiration": expiration_time,"type_filling": mt5.ORDER_FILLING_RETURN,}
                    result = mt5.order_send(close_stop_order)
                    print('A posição vendida foi zerada')
                    trades = get_trades()
                    if result.retcode != mt5.TRADE_RETCODE_DONE:
                        print("2. order_send failed, retcode={}".format(result.retcode))
                elif type_trade == 1:
                    point = mt5.symbol_info(symbol1).point
                    price = mt5.symbol_info_tick(symbol1).last
                    deviation = 20
                    close_stop_order = {
                        "action": mt5.TRADE_ACTION_DEAL,"symbol": symbol1,"volume": float(lot1),"type": mt5.ORDER_TYPE_BUY,
                        "price": price,"deviation": deviation,"magic": 234000,"comment": "Zeragem mercado","type_time": mt5.ORDER_TIME_GTC,
                        "expiration": expiration_time,"type_filling": mt5.ORDER_FILLING_RETURN,}
                    result = mt5.order_send(close_stop_order)
                    print('A posição comprada foi zerada')
                    trades = get_trades()
                    if result.retcode != mt5.TRADE_RETCODE_DONE:
                        print("2. order_send failed, retcode={}".format(result.retcode))
                print('Trade encerrado por se tratar de uma ordem stop gain')   
    except NameError:
        pass
    except KeyError:
        pass

def print_horario(mensagem):
    # Obtém o horário atual no formato HH:MM
    horario_atual = datetime.now().strftime("[%H:%M]")
    # Imprime a mensagem com o horário
    print(f"{horario_atual} {mensagem}")
    print(' ')


###############################################################################################################################################################################################################################
###############################################################################################################################################################################################################################

# ANÁLISAR ATIVO SUBJACENTE PARA INICIAR OPERAÇÕES
'''
while True:
    data_hoje = datetime.today().strftime('%Y-%m-%d')
    dados = yf.download('PETR4.SA', interval='1d')
    display(dados)
    # Última linha dos dados (último dia disponível)
    ultimo_dia = dados.iloc[-1]
    print('ultimo_dia ', ultimo_dia)
    # Verificar se o índice da última linha é igual ao dia de hoje
    data_ultimo_dia = ultimo_dia.name.date()
    if data_ultimo_dia == datetime.today().date():
        # Calcular a variação entre o preço de abertura de hoje e o preço de fechamento do último dia
        preco_abertura_hoje = dados.iloc[-1]['Open']
        preco_fechamento_ontem = dados.iloc[-2]['Close']  # Último dia disponível menos um dia
        variacao = preco_abertura_hoje - preco_fechamento_ontem
        print(f'Variação entre o preço de abertura de hoje e o preço de fechamento do dia anterior: {variacao:.2f}')
        break  # Sair do loop pois encontramos os dados de hoje
    else:
        print(f'Dados mais recentes não correspondem ao dia de hoje ({datetime.today().date()}). Aguardando atualização...')
        time.sleep(30)  # Esperar 1/2 minuto e tentar novamente
        continue
        
if variacao > 0.50 or variacao < -0.50:
    print('Dia com oscilação superior ao limite')

else:
'''
# ABRIR METATRADER
if not mt5.initialize():
    print_horario('Inicialização MT5 falhou, error code = ', mt5.last_error())
    quit()
    
stop_gain_pending = False
stop_gain_trading = False
pd.set_option('display.max_columns', None)

# ATIVOS E INFOS
symbol1,symbol2 = verificar_disponibilidade_ativos()

# CÓGIGO SERÁ EXECUTADO ATÉ QUE O HORÁRIO LIMITE PRÉ-ESTABELECIDO NÃO SEJA ATINGIDO
while not is_time_exceeded():
    symbol1_ask, symbol1_bid, symbol1_last, lot1 = ativos_parametros(symbol1,symbol2)

    # PEGAR POSIÇÕES AGENDADAS
    agendadas = get_agendadas()
    trades = get_trades()

    # ADICIONAR ORDENS AGENDADAS CASO NÃO TENHA NENHUMA        
    if len(agendadas) == 0 and len(trades) == 0:
        print_horario('Adicionando ordens no book...')
        lista_ordens = []
        symbol1_last = mt5.symbol_info_tick(symbol1).last
        for valor in range(-4, 5):
            valor_decimal = valor / 100
            if valor_decimal != 0 and valor_decimal != -0.01 and valor_decimal != 0.01:
                price = symbol1_last + valor_decimal
                price = round(price,2)
                if price != 0.01 and price != 0.00:
                    deviation = 5
                    print(price)
                    if valor_decimal > 0:
                        trade = 'Venda'
                        print_horario('Adicionando ordem de venda no book...')
                        type_to_send = mt5.ORDER_TYPE_SELL_LIMIT
                    elif valor_decimal < 0:
                        trade = 'Compra'
                        print_horario('Adicionando ordem de compra no book...')
                        type_to_send = mt5.ORDER_TYPE_BUY_LIMIT

                    agend_order = {'action': mt5.TRADE_ACTION_PENDING,'symbol': symbol1,'volume': float(lot1),'type': type_to_send,
                                'price': price,'deviation': deviation,'magic': 234000,'comment':trade,'type_time': mt5.ORDER_TIME_SPECIFIED,
                                "expiration": expiration_timestamp,'type_filling': mt5.ORDER_FILLING_RETURN,}
                    
                    agend_order = mt5.order_send(agend_order)
                    if agend_order is None:
                        print_horario(f"Erro ao enviar ordem: {mt5.last_error()}")
                    else:
                        if agend_order.retcode != mt5.TRADE_RETCODE_DONE:
                            print_horario("order_send falhou, retcode: {}".format(agend_order.retcode))
                            time.sleep(5)
                        elif agend_order.retcode == mt5.TRADE_RETCODE_DONE:
                            lista_ordens.append(agend_order)
                    time.sleep(2)
                    
        print(lista_ordens)
    
    # MONITORAMENTO ORDENS AGENDADAS
    elif len(agendadas) >= 1 and len(trades) == 0:
        verificando_agendadas = False
        spread = spread_book()
        display(spread)
        #print('STOP GAIN PARA POSIÇÃO COMPRADA',(spread.iloc[0,1]))
        #print('STOP GAIN PARA POSIÇÃO VENDIDA',(spread.iloc[1,1]))
        timezone = pytz.timezone("Etc/UTC")
        today = datetime.now(timezone)
        tomorrow = today + timedelta(days=1)
        utc_from = datetime(today.year, today.month, today.day, tzinfo=timezone)
        utc_to = datetime(tomorrow.year, tomorrow.month, tomorrow.day, tzinfo=timezone)
        ticks = mt5.copy_ticks_range(symbol2, utc_from, utc_to, mt5.COPY_TICKS_ALL)
        ticks_frame = pd.DataFrame(ticks)
        ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')
        tick_first_bid = ticks_frame['bid'].iloc[-1]
        tick_first_ask = ticks_frame['ask'].iloc[-1]
        print_horario('Monitorando ordens agendadas...')
        while len(trades) == 0 and not is_time_exceeded():
            agendadas = get_agendadas()
            symbol1_ask, symbol1_bid, symbol1_last, lot1 = ativos_parametros(symbol1,symbol2)
            # Adicionar ordens nas lacunas do book
            if symbol1_ask not in agendadas['price_open'].values and symbol1_bid not in agendadas['price_open'].values: 
                if symbol1_ask + 1 not in agendadas['price_open'].values or symbol1_bid - 1 not in agendadas['price_open'].values:
                    add_lacuna = ordem_lacuna(lista_ordens)
            # Monitorar todas as ordens agendadas
            for indice, linha in agendadas.iterrows():
                agendadas = get_agendadas()
                # Verifica se tem ordens de stop gain e trades abertos
                trades = get_trades()
                position_price_open = (linha['price_open'])
                if mt5.market_book_add(symbol1):
                    items = mt5.market_book_get(symbol1)
                    databook = pd.DataFrame(items)
                else:
                    print_horario("mt5.market_book_add failed, error code =",mt5.last_error())
                spread_sec = databook.iloc[4:6]
                tem_stop_gain = any(agendadas['comment'].str.contains('Stop Gain', case=False))
                if not tem_stop_gain:
                    stop_gain_pending = False
                    stop_gain_trading = False
    
                # SE EXISTE STOP GAIN DE ORDEM PENDENTE, VERIFICA-RÁ A QUANTIDADE NO BOOK
                if stop_gain_pending is True and stop_gain_trading is False and len(trades) == 0:
                    # CANCELAR STOP GAIN DE COMPRA
                    if spread_sec.iloc[0,2] < 70000:
                        type_order_to_cancel = int(linha['type'])
                        if type_order_to_cancel == 2:
                            cancel = cancel_stop_gain_pending(request_stop_gain)
                            print_horario('Book vazio, stop gain cancelado')
                            stop_gain_pending = False
                            
                    # CALCELAR STOP GAIN DE VENDA
                    elif spread_sec.iloc[1,2] < 70000:
                        type_order_to_cancel = int(linha['type'])                        
                        if type_order_to_cancel == 3:    
                            cancel = cancel_stop_gain_pending(request_stop_gain)
                            print_horario('Book vazio, stop gain cancelado')
                            stop_gain_pending = False
                    
                # SE NÃO EXISTE NENHUM STOP GAIN, VERIFICA BOOK PARA ADICIONAR      
                elif stop_gain_pending is False and stop_gain_trading is False and len(trades) == 0:
                    agendadas = get_agendadas()
                    if verificando_agendadas is False:
                        print_horario('Verificando stop gain de ordem pendente')
                        verificando_agendadas = True
                    symbol1_ask, symbol1_bid, symbol1_last, lot1 = ativos_parametros(symbol1,symbol2)
                    tem_stop_gain = any(agendadas['comment'].str.contains('Stop Gain', case=False))
                    if not tem_stop_gain:
                        stop_gain_pending = False
                        stop_gain_trading = False
                        
                    if stop_gain_pending is False:
                        symbol1_ask, symbol1_bid, symbol1_last, lot1 = ativos_parametros(symbol1,symbol2)
                        spread = spread_book()

                        try:
                            # ADICIONAR ORDEM STOP GAIN PARA POSIÇÃO COMPRADA
                            if spread.iloc[0,2] > 200000 and spread.iloc[1,2] > 70000:
                                #price_stop_gain = spread.iloc[0,1]
                                price_stop_gain = spread.iloc[1,1] + 0.01
                                if symbol1_ask not in agendadas['price_open'].values and symbol1_bid in agendadas['price_open'].values:
                                    request_stop_gain = add_stopgain_compra(price_stop_gain)
                                    stop_gain_pending = True
       
                            # ADICIONAR ORDEM STOP GAIN PARA POSIÇÃO VENDIDA
                            elif spread.iloc[1,2] > 200000 and spread.iloc[0,2] > 70000:
                                #price_stop_gain = spread.iloc[1,1]
                                price_stop_gain = spread.iloc[0,1] - 0.01
                                if symbol1_bid not in agendadas['price_open'].values and symbol1_ask in agendadas['price_open'].values:
                                    request_stop_gain = add_stopgain_compra(price_stop_gain)
                                    stop_gain_pending = True
                        except IndexError:
                            pass

            # CANCELAR STOP GAIN CASO O BOOK ESTIVER VAZIO E NENHUM TRADE EM ABERTO              
            agendadas = get_agendadas()
            trades = get_trades()
            # Verificar se tem ordem com comentário stop gain
            tem_stop_gain = any(agendadas['comment'].str.contains('Stop Gain', case=False))
            if tem_stop_gain:
                # Encontrar preço do stop gain 
                valor_stop_gain = agendadas.loc[agendadas['comment'] == 'Stop Gain', 'price_open'].values
                # Procurar o preço do "stop gain" nas ordens e caso o mesmo seja menor que 100k no book, cancelar stop gain
                if mt5.market_book_add(symbol1):
                    items = mt5.market_book_get(symbol1)
                    databook = pd.DataFrame(items)
                else:
                    print_horario("mt5.market_book_add failed, error code =",mt5.last_error())
                stopgain_inlocal = databook[databook[1].isin(valor_stop_gain)]
                # Se a variável não estiver vazia, será verificada a quantidade no book
                if not stopgain_inlocal.empty:
                    vol_stopgain = stopgain_inlocal.iloc[0, databook.columns.get_loc(2)]
                    if int(vol_stopgain) < 200000:
                        print_horario('Há stop gain de trade aberto mas não há trades abertos')
                        #NameError: name 'request_stop_gain' is not defined
                        if request_stop_gain is True:
                            stop_gain_pending_verificacao(request_stop_gain)
                        elif stop_gain_trading is True:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)

            # CANCELAR ORDEM PENDENTE CASO BOOK ESVAZIAR OU/E SE PREÇO VIRAR
            spread = spread_book()

            ticks = mt5.copy_ticks_range(symbol2, utc_from, utc_to, mt5.COPY_TICKS_ALL)
            ticks_frame = pd.DataFrame(ticks)
            ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')
            tick_now_bid = ticks_frame['bid'].iloc[-1]
            tick_now_ask = ticks_frame['ask'].iloc[-1]
            price_to_verify = [tick_now_bid, tick_now_ask]
            for price_to_cancel in price_to_verify:
                # Verifica se existe alguma ordem agendada cujo o preço seja igual ao preço atual da opção
                if any(order.price == price_to_cancel for order in lista_ordens):
                    try:
                        row_index = spread.index[spread.iloc[:, 1] == price_to_cancel]
                        row_data = pd.DataFrame(spread.loc[row_index[0]])
                        book_spread = int(row_data.iloc[2, 0])
                        for ordem in lista_ordens:
                            if ordem.price == price_to_cancel:
                                position_id = ordem.order
                                # Cancelamento de ordem pendente de compra
                                if tick_first_bid != tick_now_bid or tick_first_ask != tick_now_ask:
                                    if ordem.request.comment == 'Compra' and tick_now_bid < tick_first_bid and spread.iloc[1,2] < 300000:
                                        print_horario('Book vazio e opção de cima virou, a ordem agendada deve ser cancelada')
                                        position_to_cancel={
                                            "order": position_id,
                                            "action": mt5.TRADE_ACTION_REMOVE}
                                        mt5.order_send(position_to_cancel)
                                        agendadas = get_agendadas()
                                        lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                        lista_ordens = list(set(lista_ordens))
                                        stop_gain_pending_verificacao(request_stop_gain)   
            
                                    # Cancelamento de ordem pendente de venda
                                    elif ordem.request.comment == 'Venda' and tick_now_ask > tick_first_ask and spread.iloc[0,2] < 300000:
                                        print_horario('Book vazio e opção de cima virou, a ordem agendada deve ser cancelada')
                                        position_to_cancel={
                                            "order": position_id,
                                            "action": mt5.TRADE_ACTION_REMOVE}
                                        mt5.order_send(position_to_cancel)
                                        agendadas = get_agendadas()
                                        lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                        lista_ordens = list(set(lista_ordens))
                                        stop_gain_pending_verificacao(request_stop_gain)

                                elif ordem.request.comment == 'Compra' and spread.iloc[1,2] < 40000:
                                    print_horario('Book vazio, a ordem agendada deve ser cancelada')
                                    position_to_cancel={
                                        "order": position_id,
                                        "action": mt5.TRADE_ACTION_REMOVE}
                                    mt5.order_send(position_to_cancel)
                                    agendadas = get_agendadas()
                                    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                    lista_ordens = list(set(lista_ordens))
                                    stop_gain_pending_verificacao(request_stop_gain)

                                elif ordem.request.comment == 'Venda' and spread.iloc[0,2] < 40000:
                                    print_horario('Book vazio, a ordem agendada deve ser cancelada')
                                    position_to_cancel={
                                        "order": position_id,
                                        "action": mt5.TRADE_ACTION_REMOVE}
                                    mt5.order_send(position_to_cancel)
                                    agendadas = get_agendadas()
                                    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                    lista_ordens = list(set(lista_ordens))
                                    stop_gain_pending_verificacao(request_stop_gain)
                                    
                    except IndexError:
                        pass
            trades = get_trades()

    # MONITORAR TRADES
    elif len(trades) > 0:
        monitorando_trades = False
        printada = False
        trades = get_trades()
        price_open = round(trades['price_open'][0],2)
        price_current = round(trades['price_current'][0],2)
        type_trade = round(trades['type'][0],2)
        spread = spread_book()
        # Parametros para busca de informações do ativo secundário e main
        timezone = pytz.timezone("Etc/UTC")
        today = datetime.now(timezone)
        tomorrow = today + timedelta(days=1)
        utc_from = datetime(today.year, today.month, today.day, tzinfo=timezone)
        utc_to = datetime(tomorrow.year, tomorrow.month, tomorrow.day, tzinfo=timezone)
        
        # Informações ativo secundário
        ticks = mt5.copy_ticks_range(symbol2, utc_from, utc_to, mt5.COPY_TICKS_ALL)
        ticks_frame = pd.DataFrame(ticks)
        ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')
        tick_first_bid_secundaria = ticks_frame['bid'].iloc[-1]
        tick_first_ask_secundaria = ticks_frame['ask'].iloc[-1]
        
        # Informações ativo main
        ticks = mt5.copy_ticks_range(symbol1, utc_from, utc_to, mt5.COPY_TICKS_ALL)
        ticks_frame = pd.DataFrame(ticks)
        ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')
        tick_first_bid_main = ticks_frame['bid'].iloc[-1]
        tick_first_ask_main = ticks_frame['ask'].iloc[-1]
        
        while len(trades) > 0 and not is_time_exceeded():
            if monitorando_trades is False:
                monitorando_trades = True
                print_horario('Monitorando trade aberto...')

            #price_open = round(trades['price_open'][0],2)
            #price_current = round(trades['price_current'][0],2)
            
            # VERIFICAR SPREAD DA AÇÃO SECUNDÁRIA
            ticks_1 = mt5.copy_ticks_range(symbol2, utc_from, utc_to, mt5.COPY_TICKS_ALL)
            ticks_frame = pd.DataFrame(ticks_1)
            ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')
            tick_now_bid_secundaria = ticks_frame['bid'].iloc[-1]
            tick_now_ask_secundaria = ticks_frame['ask'].iloc[-1]
            
            # VERIFICAR SPREAD DA AÇÃO MAIN
            ticks_2 = mt5.copy_ticks_range(symbol1, utc_from, utc_to, mt5.COPY_TICKS_ALL)
            ticks_frame = pd.DataFrame(ticks_2)
            ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')
            tick_now_bid_main = ticks_frame['bid'].iloc[-1]
            tick_now_ask_main = ticks_frame['ask'].iloc[-1]

            # Verificar se houve alguma movimentação no spread da opção secundária
            if tick_first_bid_secundaria == tick_now_bid_secundaria and tick_first_ask_secundaria == tick_now_ask_secundaria:
                printada = False
            if tick_first_bid_secundaria != tick_now_bid_secundaria or tick_first_ask_secundaria != tick_now_ask_secundaria:
                spread = spread_book()
                spread_2 = spread_book_2()
                symbol1_ask, symbol1_bid, symbol1_last, lot1 = ativos_parametros(symbol1,symbol2)
                agendadas = get_agendadas()
                if printada == False:  # Verifica se a variável já foi printada
                    # Verificação de variáveis
                    if tick_first_bid_secundaria > tick_now_bid_secundaria:
                        print_horario('Bid da secundária caiu.')
                        print(spread_2.iloc[0,2])
                        print(spread_2.iloc[1,2])
                        print(' ')
                    if tick_first_bid_secundaria < tick_now_bid_secundaria:
                        print_horario('Bid da secundária subiu.')
                        print(spread_2.iloc[0,2])
                        print(spread_2.iloc[1,2])
                        print(' ')
                    if tick_first_bid_main > tick_now_bid_main:
                        print_horario('Bid da principal caiu')
                        print(spread.iloc[0,2])
                        print(spread.iloc[1,2])
                        print(' ')
                    if tick_first_bid_main < tick_now_bid_main:
                        print_horario('Bid da principal subiu')
                        print(spread.iloc[0,2])
                        print(spread.iloc[1,2])
                        print(' ')
                    if tick_first_ask_secundaria < tick_now_ask_secundaria:
                        print_horario('Ask da secundária subiu')
                        print(spread_2.iloc[0,2])
                        print(spread_2.iloc[1,2])
                        print(' ')
                    if tick_first_ask_secundaria > tick_now_ask_secundaria:
                        print_horario('Ask da secundária caiu')
                        print(spread_2.iloc[0,2])
                        print(spread_2.iloc[1,2])
                        print(' ')
                    if tick_first_ask_main < tick_now_ask_main:
                        print('Ask da principal subiu')
                        print(spread.iloc[0,2])
                        print(spread.iloc[1,2])
                        print(' ')
                    if tick_first_ask_main > tick_now_ask_main:
                        print('Ask da principal caiu')
                        print(spread.iloc[0,2])
                        print(spread.iloc[1,2])
                        print(' ')
                    printada = True  # Atualiza a variável de controle
                    
                """
                PREÇO MÉDIO AINDA ESTÁ OCORRENDO E ORDEM NÃO ESTÁ SENDO CANCELADA
                Adicionado os comentários
                Foi tirada a verificação de trade aberto em cada condição de stop e foi adicionado break do while
                """
                if type_trade == 0:
                    # Se as duas opções caíram e a oferta de compra (bid) do ativo 1 está menor que 300k, zerar à mercado (stop loss)
                    if tick_first_bid_secundaria > tick_now_bid_secundaria and tick_first_bid_main > tick_now_bid_main and spread.iloc[0,2] < 300000:
                        stop = buy_stop_market()
                        print_horario('Stop loss compra')
                        if stop_gain_pending is True and len(trades) == 0:
                            cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                        elif stop_gain_trading is True and len(trades) == 0:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                        break
                                
                    # Se a opção secundária caiu mas a main ainda não caiu, e o book está vazio, zerar à mercado (stop zero a zero)
                    elif tick_first_bid_secundaria > tick_now_bid_secundaria and spread.iloc[0,2] < 300000:
                        stop = buy_stop_market()
                        print_horario('Stop zero-a-zero, Preço virou')
                        if stop_gain_pending is True and len(trades) == 0:
                            cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                        elif stop_gain_trading is True and len(trades) == 0:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                        break
    
                    # Cancelar ordem pendente caso a opção secundária cair mas a main ainda não caiu (anti preço médio)
                    elif tick_first_bid_secundaria > tick_now_bid_secundaria:
                        print_horario('Bid secundária caiu, deve ser verificada a possibilidade de cancelamento de ordem.')
                        # Verifica se existe alguma ordem agendada cujo o preço seja igual ao preço atual da opção
                        if any(order.request.price == symbol1_ask for order in lista_ordens):
                            print_horario('Ordem pendente deve ser cancelada, sujeito a preço médio.')
                            #try:
                            print('spread')
                            print(spread)
                            # Encontra o índice igual a symbol1_last
                            row_index = spread.index[spread.iloc[:, 1] == symbol1_last]
                            print('row_index')
                            print(row_index)
                            # Encontra a linha
                            row_data = pd.DataFrame(spread.loc[row_index[0]])
                            print('row_data')
                            print(row_data)
                            book_spread = int(row_data.iloc[2, 0])
                            for ordem in lista_ordens:
                                if ordem.request.price == symbol1_last:
                                    position_id = ordem.order
                                    position_to_cancel={
                                        "order": position_id,
                                        "action": mt5.TRADE_ACTION_REMOVE}
                                    mt5.order_send(position_to_cancel)
                                    agendadas = get_agendadas()
                                    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                    lista_ordens = list(set(lista_ordens))
                                    stop_gain_trading_verificacao(stopgain_opendtrade)
                                    agendadas = get_agendadas()
                                    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                    lista_ordens = list(set(lista_ordens))
                                    if stop_gain_pending is True and len(trades) == 0:
                                        cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                                    elif stop_gain_trading is True and len(trades) == 0:
                                        cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                                    print_horario('Ordem pendente cancelada, sujeito a preço médio.')
                            #except IndexError:
                                #pass
                    # Encerrar trade caso book esvaziar
                    elif spread.iloc[0,2] < 50000:
                        stop = buy_stop_market()
                        print_horario('Stop, book vazio')
                        if stop_gain_pending is True and len(trades) == 0:
                            cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                        elif stop_gain_trading is True and len(trades) == 0:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                        break
                        
                if type_trade == 1:
                    # Se as duas opções subiram e as oferta de venda (ask) estão menores que 300k para zerar à mercado (stop loss)
                    if tick_first_ask_secundaria < tick_now_ask_secundaria and tick_first_ask_main < tick_now_ask_main and spread.iloc[1,2] < 300000:
                        stop = sell_stop_market()
                        print_horario('Stop loss venda')
                        if stop_gain_pending is True and len(trades) == 0:
                            cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                        elif stop_gain_trading is True and len(trades) == 0:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                        break
                        
                    # Se a opção secundária subiu mas a main ainda não subiu, e o book está vazio, zerar à mercado (stop zero a zero)
                    elif tick_first_ask_secundaria < tick_now_ask_secundaria and spread.iloc[1,2] < 300000:
                        print_horario('Stop zero-a-zero, Preço virou')
                        stop = sell_stop_market()
                        if stop_gain_pending is True and len(trades) == 0:
                            cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                        elif stop_gain_trading is True and len(trades) == 0:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                        break
                        
                    # Cancelar ordem pendente caso a opção secundária subir mas a main ainda não subiu (anti preço médio)
                    elif tick_first_ask_secundaria < tick_now_ask_secundaria:
                        print_horario('Ask secundária caiu, deve ser verificada a possibilidade de cancelamento de ordem.')
                        # Verifica se existe alguma ordem agendada cujo o preço seja igual ao preço atual da opção
                        if any(order.request.price == symbol1_ask for order in lista_ordens):
                            print_horario('Ordem pendente deve ser cancelada, sujeito a preço médio.')
                            #try:
                            print('spread')
                            print(spread)
                            # Encontra o índice igual a symbol1_last
                            row_index = spread.index[spread.iloc[:, 1] == symbol1_last]
                            print('row_index')
                            print(row_index)
                            # Encontra a linha
                            row_data = pd.DataFrame(spread.loc[row_index[0]])
                            print('row_data')
                            print(row_data)
                            book_spread = int(row_data.iloc[2, 0])
                            for ordem in lista_ordens:
                                if ordem.request.price == symbol1_last:
                                    position_id = ordem.order
                                    position_to_cancel={
                                        "order": position_id,
                                        "action": mt5.TRADE_ACTION_REMOVE}
                                    mt5.order_send(position_to_cancel)
                                    agendadas = get_agendadas()
                                    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                    lista_ordens = list(set(lista_ordens))
                                    stop_gain_trading_verificacao(stopgain_opendtrade)
                                    agendadas = get_agendadas()
                                    lista_ordens = [ordem for ordem in lista_ordens if ordem.request.price in agendadas['price_open'].values]
                                    lista_ordens = list(set(lista_ordens))
                                    if stop_gain_pending is True and len(trades) == 0:
                                        cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                                    elif stop_gain_trading is True and len(trades) == 0:
                                        cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                                    print_horario('Ordem pendente cancelada, sujeito a preço médio.')
                            #except IndexError:
                                #pass
            
                    # Encerrar trade caso book esvaziar
                    elif spread.iloc[1,2] < 50000:
                        stop = sell_stop_market()
                        print_horario('Stop, book vazio')
                        if stop_gain_pending is True and len(trades) == 0:
                            cancel_function_pending = cancel_stop_gain_pending(request_stop_gain)
                        elif stop_gain_trading is True and len(trades) == 0:
                            cancel_function_trading = cancel_stop_gain_trading(stopgain_opendtrade)
                        break
                        
            elif stop_gain_pending is False and stop_gain_trading is False:
                # ADICIONAR STOP GAIN PARA TRADE DE COMPRA ABERTO
                if type_trade == 0:
                    deviation = 5
                    price_stop_gain_sec = trades.iloc[0, 4] + 0.01
                    stopgain_opendtrade = {'action': mt5.TRADE_ACTION_PENDING,'symbol': symbol1,'volume': float(lot1),'type': mt5.ORDER_TYPE_SELL_LIMIT,
                                'price': price_stop_gain_sec,'deviation': deviation,'magic': 234000,'comment': 'Stop Gain',
                                'type_time': mt5.ORDER_TIME_GTC,
                                'type_filling': mt5.ORDER_FILLING_RETURN,}
                    stopgain_opendtrade = mt5.order_send(stopgain_opendtrade)
                    if stopgain_opendtrade.retcode != mt5.TRADE_RETCODE_DONE:
                        print("2. order_send failed, retcode={}".format(stopgain_opendtrade.retcode))
                        result_dict = stopgain_opendtrade._asdict()
                        for field in result_dict.keys():
                            print("   {}={}".format(field,result_dict[field]))
                            if field=="request":
                                traderequest_dict=result_dict[field]._asdict()
                                for tradereq_filed in traderequest_dict:
                                    print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                    stop_gain_trading = True
                    print_horario('Trade aberto, stop gain adicionado')

                # ADICIONAR STOP GAIN PARA TRADE DE VENDA ABERTO
                elif type_trade == 1:
                    deviation = 5
                    try:
                        # Caso não exista, adicionar stop gain 0.01 acima do mesmo preço de compra
                        price_stop_gain_sec = trades.iloc[0, 4] - 0.01
                        stopgain_opendtrade = {'action': mt5.TRADE_ACTION_PENDING,'symbol': symbol1,'volume': float(lot1),'type': mt5.ORDER_TYPE_BUY_LIMIT,
                                    'price': price_stop_gain_sec,'deviation': deviation,'magic': 234000,'comment': 'Stop Gain',
                                    'type_time': mt5.ORDER_TIME_GTC,'type_filling': mt5.ORDER_FILLING_RETURN,}
                        stopgain_opendtrade = mt5.order_send(stopgain_opendtrade)
                        if stopgain_opendtrade.retcode != mt5.TRADE_RETCODE_DONE:
                            print("2. order_send failed, retcode={}".format(stopgain_opendtrade.retcode))
                            result_dict = stopgain_opendtrade._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        stop_gain_trading = True
                        print_horario('Trade aberto, stop gain adicionado')
                    except IndexError:
                        pass

            agendadas = get_agendadas()
            trades = get_trades()

            # Verificar se tem stop gain
            tem_stop_gain = any(agendadas['comment'].str.contains('Stop Gain', case=False))
            if not tem_stop_gain:
                stop_gain_pending = False
                stop_gain_trading = False

            # STOPAR CASO TRADE ABERTO SEJA STOP GAIN
            stop_gain_trading_stop()

# CANCELAMENTO DAS ORDENS E ENCERRAMENTO DO CÓDIGO
encerramento_pregao(lista_ordens,stopgain_opendtrade)

Ambos os ativos estão visíveis
[15:33] Adicionando ordens no book...
 
0.07
[15:33] Adicionando ordem de compra no book...
 
[15:33] order_send falhou, retcode: 10022
 


KeyboardInterrupt: 